In [1]:
import numpy as np
import pandas as pd
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import calibration_and_holdout_data, summary_data_from_transaction_data
from lifetimes.plotting import plot_calibration_purchases_vs_holdout_purchases, plot_period_transactions
from datetime import datetime, timedelta
import matplotlib.pyplot as plt


In [2]:
g_datafolder = '/development/data'
g_customer_dataset = '{}/olist_customers_dataset.csv'.format(g_datafolder)
g_orders_dataset = '{}/olist_orders_dataset.csv'.format(g_datafolder)
g_payments_dataset = '{}/olist_order_payments_dataset.csv'.format(g_datafolder)
g_orderitems_dataset = '{}/olist_order_items_dataset.csv'.format(g_datafolder)


In [149]:
customer_df = load_dataset(g_customer_dataset)
parse_dates = ['order_purchase_timestamp', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
orders_df = load_dataset(g_orders_dataset, parse_dates=parse_dates)
payments_df = load_dataset(g_payments_dataset)
orderitems_df = load_dataset(g_orderitems_dataset)
cust_ord_df = orders_df.set_index('customer_id').join(customer_df.set_index('customer_id'), how="inner").reset_index()
cust_ord_df = cust_ord_df.set_index('order_id').join(orderitems_df.set_index('order_id'), how="inner").reset_index()
cust_ord_df['monetary_value'] = np.round(cust_ord_df['price'] + cust_ord_df['freight_value'], 0)
cust_ord_df['cancelled'] = 0.0
cust_ord_df['cancelled'][cust_ord_df.order_status == 'canceled'] = 1.0
customer_id_col='customer_unique_id'
datetime_col='order_purchase_timestamp'
monetary_value_col='total_value' 
calibration_period_end = datetime(2018,4,1)
observation_period_end = cust_ord_df.order_purchase_timestamp.max()


In [174]:
def load_dataset(datafile, parse_dates=None):
    df = pd.read_csv(datafile, delimiter=',', parse_dates=parse_dates)
    return df

def add_datediffcolumn(df, col_dt1, col_dt2, colname):
    df[colname] = np.round((df[col_dt1] - df[col_dt2]).dt.days + 1.0, 0)
    #df[colname] = np.round((df[col_dt1] - df[col_dt2]) / np.timedelta64(1, 'M'), 0)
    df[colname][df[colname].isna()] = 0.0
    return df

def get_calibration_holdout_data(df
                               , customer_id_col='id'
                               , datetime_col='date'
                               , calibration_period_end=None
                               , observation_period_end=None
                               , monetary_value_col='value'
                               , covariates=None):
    start = df[datetime_col].min()
    df['obs'] = 0
    df['obs'][df[datetime_col] > calibration_period_end] = 1
    sort_cols = [customer_id_col, 'obs', datetime_col]
    df['x'] = df.sort_values(sort_cols).groupby([customer_id_col, 'obs']).cumcount()+1
    df['x'] = df['x'] - 1
    df['first'] = df[datetime_col]       
    df['last'] = df[datetime_col]       
    groupby_cols = [customer_id_col, 'obs']
    all_cols = groupby_cols + [datetime_col, monetary_value_col, 'x', 'first', 'last'] + covariates
    agg_map = {monetary_value_col:'last', 'x':'max', 'first':'min', 'last':'max' }
    for covariate in covariates:
        agg_map[covariate] = 'sum'
    df = df.sort_values(sort_cols)[all_cols].groupby(groupby_cols).agg(agg_map).reset_index()
    df['endobs'] = calibration_period_end    
    df['endobs'][df.obs == 1] = observation_period_end
    df = add_datediffcolumn(df, 'last', 'first', 't')
    df = add_datediffcolumn(df, 'endobs', 'first', 'T')
    cols = ['x', 't', 'T', monetary_value_col] + covariates
    cal_df = df[df.obs == 0][[customer_id_col] + cols] 
    cal_df.columns = [customer_id_col] + ['{}_cal'.format(colname) for colname in cols]
    hold_df = df[df.obs == 1][[customer_id_col] + cols] 
    hold_df.columns = [customer_id_col] + ['{}_holdout'.format(colname) for colname in cols]
    df = cal_df.set_index(customer_id_col).join(hold_df.set_index(customer_id_col), how="left").reset_index().fillna(0.0)
    return df


In [177]:
df = get_calibration_holdout_data(cust_ord_df
                                , customer_id_col=customer_id_col
                                , datetime_col=datetime_col
                                , calibration_period_end=calibration_period_end
                                , observation_period_end=observation_period_end
                                , monetary_value_col='monetary_value'
                                , covariates=['cancelled'])

In [181]:
df[df.cancelled_cal > 0].head(n=20)


,customer_unique_id,x_cal,t_cal,T_cal,monetary_value_cal,cancelled_cal,x_holdout,t_holdout,T_holdout,monetary_value_holdout,cancelled_holdout
88,0058f300f57d7b93c477a131a59b36c3,1,32.0,41.0,96.0,1.0,0.0,0.0,0.0,0.0,0.0
311,013f66477aa3210eb05fec3fa184de33,0,1.0,69.0,116.0,1.0,0.0,0.0,0.0,0.0,0.0
476,01ea7dfdac01a4e8fbe2902b73510b20,1,1.0,51.0,62.0,1.0,0.0,0.0,0.0,0.0,0.0
628,028ccf85a88eb595cfa50d7a0d417868,0,1.0,63.0,167.0,1.0,0.0,0.0,0.0,0.0,0.0
920,03b2cffef7bf1140a4dc136a673d6b11,0,1.0,202.0,69.0,1.0,0.0,0.0,0.0,0.0,0.0
1206,04eac1171d6846c64e71bf03b3e4174e,0,1.0,61.0,90.0,1.0,0.0,0.0,0.0,0.0,0.0
1289,05378bc3a8e9dd6da066418c0e156a0c,0,1.0,219.0,78.0,1.0,0.0,0.0,0.0,0.0,0.0
1320,055ec572ac7f3c7bdd04a183830ebe59,1,31.0,287.0,2027.0,1.0,0.0,0.0,0.0,0.0,0.0
1384,05a3b50193563913613acada5c86b1d2,0,1.0,287.0,123.0,1.0,0.0,0.0,0.0,0.0,0.0
1506,062fe2e9df8c4c7b058a6d711bae0f3d,0,1.0,43.0,57.0,1.0,0.0,0.0,0.0,0.0,0.0


In [173]:
cust_ord_df[cust_ord_df.customer_unique_id == '394ac4de8f3acb14253c177f0e15bc58'].sort_values(['obs', 'x']).head(n=20)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,seller_id,shipping_limit_date,price,freight_value,monetary_value,cancelled,obs,x,first,last
42182,60114243c9bedbdafe9bc312bd608936,9340fee89818bd84d3a005f8fbfe5b82,delivered,2017-12-22 19:23:43,2017-12-22 19:35:19,2017-12-26 20:15:14,2017-12-27 16:15:46,2018-01-12,394ac4de8f3acb14253c177f0e15bc58,1239,...,e9779976487b77c6d4ac45f75ec7afe9,2017-12-29 19:35:19,139.56,8.72,148.0,0.0,0,0,2017-12-22 19:23:43,2017-12-22 19:23:43
23554,35b6b206735a826adef009d685f69b16,e73f4aab3c918bbce64b1090c5802120,delivered,2018-03-05 19:53:08,2018-03-05 20:40:23,2018-03-09 00:35:23,2018-03-09 17:16:24,2018-03-15,394ac4de8f3acb14253c177f0e15bc58,1239,...,4ebdc7e6cd6102a022dadc49156d4ea8,2018-03-09 20:32:04,14.90,8.40,23.0,0.0,0,1,2018-03-05 19:53:08,2018-03-05 19:53:08
33920,4cc8845d2a687fdd3d47898857cecdfd,db846bcd3ea0ef56effb856bd0266a90,delivered,2018-03-15 17:06:45,2018-03-15 17:15:53,2018-03-16 20:33:33,2018-03-26 15:29:08,2018-04-06,394ac4de8f3acb14253c177f0e15bc58,1239,...,2a5b78b41cd05baeac8df54c6606b92c,2018-03-21 17:15:53,145.00,18.89,164.0,0.0,0,2,2018-03-15 17:06:45,2018-03-15 17:06:45
72671,a5b36a263c9692d52e2243b9ef05c6d6,7f25ffaada2024d3e809765b76d3507a,delivered,2018-06-20 12:34:14,2018-06-20 12:58:07,2018-06-21 10:05:00,2018-06-25 14:26:38,2018-07-16,394ac4de8f3acb14253c177f0e15bc58,1239,...,a5a1bfcf728ab0e19182959cf0771ee4,2018-06-26 12:58:07,119.70,13.50,133.0,0.0,1,0,2018-06-20 12:34:14,2018-06-20 12:34:14
25654,3a4e65de6ffd941f6a4ee824d472aabb,ba7df02cddb36d302d9a87ed492fdf07,delivered,2018-08-15 18:01:23,2018-08-15 18:40:07,2018-08-16 15:37:00,2018-08-17 19:18:48,2018-09-03,394ac4de8f3acb14253c177f0e15bc58,1239,...,2a5b78b41cd05baeac8df54c6606b92c,2018-08-23 18:31:40,145.00,41.14,186.0,0.0,1,1,2018-08-15 18:01:23,2018-08-15 18:01:23
25655,3a4e65de6ffd941f6a4ee824d472aabb,ba7df02cddb36d302d9a87ed492fdf07,delivered,2018-08-15 18:01:23,2018-08-15 18:40:07,2018-08-16 15:37:00,2018-08-17 19:18:48,2018-09-03,394ac4de8f3acb14253c177f0e15bc58,1239,...,f25e239052084705e17a982bc600ab2a,2018-08-21 18:31:40,69.00,21.60,91.0,0.0,1,2,2018-08-15 18:01:23,2018-08-15 18:01:23


In [48]:
df[df.customer_unique_id == '004288347e5e88a27ded2bb23747066c'].head(n=20)

,customer_unique_id,obs,total_value,x,first,last,start,end
106,004288347e5e88a27ded2bb23747066c,0,354.0,1.0,2017-07-27 14:13:03,2018-01-14 07:36:54,2016-09-04 21:15:19,2018-04-01


In [8]:
cust_ord_df.order_status.unique()

array(['delivered', 'shipped', 'canceled', 'invoiced', 'processing',
       'approved', 'unavailable'], dtype=object)

In [ ]:
customer_df.customer_state.unique()

In [5]:
observation_period_end = cust_ord_df.order_purchase_timestamp.max()
calibration_period_end = datetime(2018,4,1)
data_df = calibration_and_holdout_data(cust_ord_df
                                    , customer_id_col='customer_unique_id'
                                    , datetime_col='order_purchase_timestamp'
                                    , calibration_period_end=calibration_period_end
                                    , observation_period_end=observation_period_end
                                    , monetary_value_col='total_value'
                                    , covariates=['cancelled'])

In [6]:
data_df.head()

,frequency_cal,recency_cal,T_cal,monetary_value_cal,frequency_holdout,monetary_value_holdout,duration_holdout
customer_unique_id,,,,,,,
0000f46a3911fa3c0805444483337064,0.0,0.0,387.0,0.0,0.0,0.0,155
0000f6ccb0745a6a4b88665a16c9f078,0.0,0.0,171.0,0.0,0.0,0.0,155
0004aac84e0df4da2b147fca70cf8255,0.0,0.0,138.0,0.0,0.0,0.0,155
00053a61a98854899e70ed204dd4bafe,0.0,0.0,32.0,0.0,0.0,0.0,155
0005e1862207bf6ccc02e4228effd9a0,0.0,0.0,393.0,0.0,0.0,0.0,155


## Implementation

In [ ]:
len(data_df)

In [ ]:
# x ==> number of repeat purchases
# t ==> First purchase to last purchase
# T ==> First purchase to end of observation period


In [194]:
# Setup Regressors (Covariates) for location of 1st-stage prior, i.e. beta = [log(lambda), log(mu)]
def set_regressors(data, covariates=[]):
    data['intercept'] = 1.0       
    covars = np.matrix(data[['intercept'] + covariates])
    K = len(covariates)
    return covars, K

def get_diag(shape, val):
    d = np.zeros(shape=shape)
    np.fill_diagonal(d, val) 
    return d

# set hyper priors "log_lambda", "log_mu"
def set_hyperpriors(K):  
    beta_0 = np.zeros(shape=(K, 2))
    A_0 = get_diag(shape=(K, K), val=0.01) # diffuse precision matrix
    # set diffuse hyper-parameters for 2nd-stage prior of gamma_0; follows defaults from rmultireg example
    nu_00 = 3 + K  # 30
    gamma_00 = get_diag(shape=(2, 2), val=nu_00) # diffuse precision matrix
    hyper_prior = {'beta_0': beta_0, 'A_0':A_0, 'nu_00':nu_00, 'gamma_00':gamma_00}
    return hyper_prior

def run_single_chain(data, hyper_prior):
    ## initialize arrays for storing draws ##
    nr_of_cust <- len(data)
    nr_of_draws <- (mcmc - 1) %/% thin + 1

    level_1_draws <- array(NA_real_, dim = c(nr_of_draws, 4, nr_of_cust))
    dimnames(level_1_draws)[[2]] <- c("lambda", "mu", "tau", "z")

    level_2_draws <- array(NA_real_, dim = c(nr_of_draws, 2 * K + 3))
    nm <- c("log_lambda", "log_mu")
    if (K > 1)
      nm <- paste(rep(nm, times = K), rep(colnames(covars), each = 2), sep = "_")
    dimnames(level_2_draws)[[2]] <- c(nm, "var_log_lambda", "cov_log_lambda_log_mu", "var_log_mu")

    ## initialize parameters ##

    level_1 <- level_1_draws[1, , ] # nolint
    level_1["lambda", ] <- mean(data$x) / mean(ifelse(data$t.x == 0, data$T.cal, data$t.x))
    level_1["mu", ] <- 1 / (data$t.x + 0.5 / level_1["lambda", ])

    ## run MCMC chain ##

    hyper_prior$beta_0[1, "log_lambda"] <- log(mean(level_1["lambda", ]))
    hyper_prior$beta_0[1, "log_mu"] <- log(mean(level_1["mu", ]))

    for (step in 1:(burnin + mcmc)) {
      if (step %% trace == 0)
        cat("chain:", chain_id, "step:", step, "of", (burnin + mcmc), "\n")

      # draw individual-level parameters
      level_1["z", ] <- draw_z(data, level_1)
      level_1["tau", ] <- draw_tau(data, level_1)

      level_2 <- draw_level_2(covars, level_1, hyper_prior)

      draw <- draw_level_1(data, covars, level_1, level_2)
      level_1["lambda", ] <- draw$lambda
      level_1["mu", ] <- draw$mu

      # store
      if ( (step - burnin) > 0 & (step - 1 - burnin) %% thin == 0) {
        idx <- (step - 1 - burnin) %/% thin + 1
        level_1_draws[idx, , ] <- level_1 # nolint
        level_2_draws[idx, ] <- c(level_2$beta, level_2$gamma[1, 1], level_2$gamma[1, 2], level_2$gamma[2,
          2])
      }
    }

    # convert MCMC draws into coda::mcmc objects
    return(list(
      "level_1" = lapply(1:nr_of_cust,
                         function(i) mcmc(level_1_draws[, , i], start = burnin, thin = thin)), # nolint
      "level_2" = mcmc(level_2_draws, start = burnin, thin = thin)))
    


In [195]:
covars, K = set_regressors(df, covariates=['cancelled_cal'])
hyper_prior = set_hyperpriors(K)


In [186]:
run_single_chain(df, hyper_prior=hyper_prior)

(63881, 2)

In [196]:
hyper_prior

{'beta_0': array([[0., 0.]]),
 'A_0': array([[0.01]]),
 'nu_00': 4,
 'gamma_00': array([[4., 0.],
        [0., 4.]])}

In [197]:
K

1

In [199]:
len(df)

63881